# Numerical PDE I: Finite Difference

## Introduction

Partial Differential Equations (PDEs) are fundamental tools in the
mathematical modeling of various physical phenomena.
Unlike Ordinary Differential Equations (ODEs), which involve functions
of a single variable and their derivatives, PDEs involve functions of
multiple variables and their partial derivatives.
This distinction makes PDEs particularly powerful in describing
systems where changes occur in more than one dimension, such as in
space and time.

### What are PDEs?

A PDE is an equation that relates the partial derivatives of a
multivariable function.
In general form, a PDE can be written as:
\begin{align}
  F\left(x_1, x_2, \ldots, x_n,
         u,
	 \frac{\partial   u}{\partial x_1}, \frac{\partial u}{\partial x_2}, \ldots,
	 \frac{\partial^k u}{\partial x_1^{k_1}\partial x_2^{k_2} \ldots \partial x_n^{k_n}}\right) = 0
\end{align}
where $u = u(x_1, x_2, \ldots, x_n)$ is the unknown function, and
$\partial u/\partial x_i$ denotes the partial derivatives of $u$ with
respect to the variables $x_i$.

PDEs are essential in modeling continuous systems where the state of
the system depends on multiple variables.
They appear in various fields such as physics, engineering, finance,
and biology, describing phenomena like heat conduction, wave
propagation, fluid dynamics, and quantum mechanics.

### Definition and Significance in Modeling Continuous Systems

PDEs provide a framework for formulating problems involving functions
of several variables and their rates of change.
They are indispensable in describing the behavior of physical systems
where spatial and temporal variations are intrinsic.
Examples include:
* Advection Equation:
  Models a quantity, e.g., density, moves with velocity $c$:
  \begin{align}
    \frac{\partial u}{\partial t} + c \nabla u = 0.
  \end{align}
* Heat Equation:
  Models the distribution of heat (or temperature) in a given region
  over time.
  \begin{align}
    \frac{\partial u}{\partial t} = \alpha \nabla^2 u.
  \end{align}
* Wave Equation:
  Describes the propagation of waves, such as sound or electromagnetic
  waves, through a medium.
  \begin{align}
  \frac{\partial^2 u}{\partial t^2} = c^2 \nabla^2 u
  \end{align}
* Laplace's Equation:
  Represents steady-state solutions where the system does not change
  over time, such as electric potential in a region devoid of charge.
  \begin{align}
  \nabla^2 u = 0
  \end{align}

The ability to model such diverse phenomena underscores the
versatility and importance of PDEs in scientific and engineering
disciplines.

## Finite Difference Methods (FDM)

Solving Partial Differential Equations (PDEs) numerically is essential
for modeling complex physical systems that lack closed-form analytical
solutions.
Among the various numerical methods available, Finite Difference
Methods (FDM) are particularly popular due to their simplicity and
ease of implementation.
They approximate the derivatives in PDEs by using difference on a
discretized grid.
This approach transforms continuous PDEs into discrete algebraic
equations that can be solved iteratively.
FDM is widely used in engineering and scientific computations due to
its straightforward application to regular grids and its compatibility
with existing numerical solvers.

### Forward Time Centered Space

The Forward Time Centered Space (FTCS) scheme is one of the simplest
explicit finite difference methods used to solve time-dependent PDEs.
It approximates the time derivative using a forward difference and the
spatial derivatives using centered differences.

Consider the linear advection equation that models the transport of a
quantity $u$ with constant speed $c$:
\begin{align}
  \frac{\partial u}{\partial t} + c \frac{\partial u}{\partial x} = 0
\end{align}

Let $u_i^n$ denote the numerical approximation of $u$ at spatial index
$i$ and time level $n$.
The FTCS scheme discretizes the advection equation as follows:
\begin{align}
  \frac{u_i^{n+1} - u_i^n}{\Delta t} + c \frac{u_{i+1}^n - u_{i-1}^n}{2 \Delta x} = 0
\end{align}

Solving for $u_i^{n+1}$:
\begin{align}
  u_i^{n+1} = u_i^n - \frac{c \Delta t}{2 \Delta x} \left( u_{i+1}^n - u_{i-1}^n \right)
\end{align}

This explicit update rule allows the computation of the solution at
the next time step based on the current and neighboring spatial
points.


Here is a simple python implementation:

In [ ]:
# Parameters

c  = 1.0    # advection speed
l  = 1.0    # domain size
dt = 0.001  # time step

nx = 101    # number of spatial points
nt = 1000   # number of time steps

In [ ]:
import numpy as np

X, dx = np.linspace(0, l, nx, retstep=True)  # spatial grid
U0    = np.sin(2*np.pi * X)                  # initial condition: sinusoidal wave

In [ ]:
# Forward Time Centered Space (FTCS) scheme

def FTCS(c, U0, dx, dt, n):
    U = [U0]
    for _ in range(n):
        U0 = U[-1]
        U1 = U0 - (c*dt) / (2*dx) * (np.roll(U0,-1) - np.roll(U0,1))
        U.append(U1)
    return np.array(U)

In [ ]:
U     = np.sin(2*np.pi * (X - c*dt*nt)) # analytical solution
UFTCS = FTCS(c, U0, dx, dt, nt)         # numerical solution

Let's now plot the result!
After $t = dt n_t = 1$, the solution should match the initial
condition exactly.

In [ ]:
from matplotlib import pyplot as plt

plt.plot(X, U0,              label='Initial Condition')
plt.plot(X, U, ':',          label='Exact Solution')
plt.plot(X, UFTCS[-1], '.-', label='FTCS Scheme')
plt.xlabel('x')
plt.ylabel('u')
plt.legend()

However, the numerical solution is oscillating.
This looks like a numerical artifact.
Let's inspect it with a movie.

In [ ]:
from matplotlib.animation import ArtistAnimation
from IPython.display import HTML
from tqdm import tqdm

def animate(X, U):
    fig, ax = plt.subplots(1,1)
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    frames = []
    for n in tqdm(range(len(U))):
        f = ax.plot(X, U[n], 'C0.-', animated=True)
        frames.append(f)
        plt.close()
    
    return ArtistAnimation(fig, frames, interval=50)

In [ ]:
anim = animate(X, UFTCS)

HTML(anim.to_html5_video())  # display animation
# anim.save('FTCS.mp4')        # save animation